In [ ]:
import sys
import importlib as imp
import importlib as imp

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn import metrics

import experiment_settings
import file_methods, plots, data_processing
from DIRECTORIES import PREDICTIONS_DIRECTORY

savefig_dpi = 300

In [ ]:
print(f"python version = {sys.version}")
print(f"numpy version = {np.__version__}")
print(f"xarray version = {xr.__version__}")
print(f"tensorflow version = {tf.__version__}")

In [ ]:
def get_percentiles(label,preds,th=5):
    p5, p95 = np.percentile(label,(th,100-th))
    
    idx5  = (label<=p5 )
    idx95 = (label>=p95)

    return label[idx5], preds[idx5], label[idx95], preds[idx95]

def mean_error(labels,preds):
    return np.mean(labels-preds)

In [ ]:
EXP_NAME = "MPI_IV_hpt00"
percentile_th = 5

imp.reload(data_processing)
imp.reload(experiment_settings)
imp.reload(file_methods)

NUM_HPT = 9
MSE = np.empty((NUM_HPT,5))
MAE = np.empty((NUM_HPT,5))
MXE  = np.empty((NUM_HPT,5))
ME  = np.empty((NUM_HPT,5))
R2  = np.empty((NUM_HPT,5))

MSE_5 = np.empty((NUM_HPT,5))
MAE_5 = np.empty((NUM_HPT,5))
MXE_5  = np.empty((NUM_HPT,5))
ME_5  = np.empty((NUM_HPT,5))
R2_5  = np.empty((NUM_HPT,5))

MSE_95 = np.empty((NUM_HPT,5))
MAE_95 = np.empty((NUM_HPT,5))
MXE_95  = np.empty((NUM_HPT,5))
ME_95  = np.empty((NUM_HPT,5))
R2_95  = np.empty((NUM_HPT,5))

for hpt in range(NUM_HPT):
    print("*********  "+EXP_NAME+str(hpt)+"  *********")

    # Get model settings
    settings = experiment_settings.get_settings(EXP_NAME+str(hpt))

    # Load predictions
    for rng_seed in settings["rng_seed_list"]:
        settings["rng_seed"] = rng_seed
        model_name = file_methods.get_model_name(settings)
        predictions = file_methods.load_predictions(PREDICTIONS_DIRECTORY + model_name + "_predictions.pickle")

        label5, preds5, label95, preds95 = get_percentiles(predictions["labels_val"], predictions["pred_val"], th=percentile_th)

        # Metrics (Validation, testing, OOS)
        MSE[hpt,rng_seed] = metrics.mean_squared_error(
                predictions["labels_val"], predictions["pred_val"]
            )
        MAE[hpt,rng_seed] = metrics.mean_absolute_error(
                predictions["labels_val"], predictions["pred_val"]
            )
        MXE[hpt,rng_seed] = metrics.max_error(
                predictions["labels_val"], predictions["pred_val"]
            )
        ME[hpt,rng_seed] = mean_error(
                predictions["labels_val"], predictions["pred_val"]
            )
        R2[hpt,rng_seed] = metrics.r2_score(
                predictions["labels_val"], predictions["pred_val"]
            )
        
        MSE_5[hpt,rng_seed] = metrics.mean_squared_error(
                label5, preds5
            )
        MAE_5[hpt,rng_seed] = metrics.mean_absolute_error(
                label5, preds5
            )
        MXE_5[hpt,rng_seed] = metrics.max_error(
                label5, preds5
            )
        ME_5[hpt,rng_seed] = mean_error(
                label5, preds5
            )
        R2_5[hpt,rng_seed] = metrics.r2_score(
                label5, preds5
            )
        
        MSE_95[hpt,rng_seed] = metrics.mean_squared_error(
                label95, preds95
            )
        MAE_95[hpt,rng_seed] = metrics.mean_absolute_error(
                label95, preds95
            )
        MXE_95[hpt,rng_seed] = metrics.max_error(
                label95, preds95
            )
        ME_95[hpt,rng_seed] = mean_error(
                label95, preds95
            )
        R2_95[hpt,rng_seed] = metrics.r2_score(
                label95, preds95
            )
        print("Seed "+str(rng_seed)+":  "\
            +"mse = "+plots.num_lab(MSE[hpt,rng_seed],3)\
            +", mae = "+plots.num_lab(MAE[hpt,rng_seed],3)\
            +", me = "+plots.num_lab(MXE[hpt,rng_seed],3)\
            +", r2 = "+plots.num_lab(R2[hpt,rng_seed],3)\
        )
        

In [ ]:
lbls = ["hpt"+str(ii) for ii in range(1,NUM_HPT+1)]

RMSE = np.sqrt(MSE)
RMSE_5 = np.sqrt(MSE_5)
RMSE_95 = np.sqrt(MSE_95)

exp_num = np.arange(NUM_HPT)
PLOT = np.empty(5,dtype='object')

fig, axs = plt.subplots(2,3,figsize=(20, 13))
for qt in range(3):
    if qt==0:
        PLOT = np.array([RMSE, MAE, R2, MXE, ME])
        shift = 0.0
        clr = 'k'
    elif qt==1:
        PLOT = np.array([RMSE_5, MAE_5, R2_5, MXE_5, ME_5])
        shift = 0.15
        clr = 'r'
    elif qt==2:
        PLOT = np.array([RMSE_95, MAE_95, R2_95, MXE_95, ME_95])
        shift =-0.15
        clr = 'g'


    for jj in range(5):
        ax = plt.subplot(2, 3, jj+1)
        for ii in range(5):
            ax.scatter(exp_num+shift,PLOT[jj,:,ii],color=clr,s=15)
        ax.scatter(3+shift,PLOT[jj,3,3],marker='*',color=clr,s=100)

        for val in exp_num:
            ax.plot([val+0.5,val+0.5],[-10,10],color='gray',alpha=0.1)

        if qt==0:
            ax.set_xlim(-0.5,val+0.5)
            ax.set_xticks(range(NUM_HPT))
            ax.set_xticklabels(lbls, rotation=45,fontsize=14)
            if jj == 0:
                ax.set_ylabel("Root mean squared error [W/m$^2$]",fontsize=14)
                ax.set_ylim(0.1,0.24)
                ax.set_title("(a)",fontsize=15)
            elif jj == 1:
                ax.set_ylabel("Mean absolute error [W/m$^2$]",fontsize=14)
                ax.set_ylim(0.1,0.24)
                ax.set_title("(b)",fontsize=15)
            elif jj == 2:
                ax.set_ylabel("R$^2$",fontsize=14)
                ax.set_ylim(-6,1)
                ax.set_title("(c)",fontsize=15)
            elif jj == 3:
                ax.set_ylabel("Maximum absolute error [W/m$^2$]",fontsize=14)
                ax.set_ylim(0.3,1.1)
                ax.set_title("(d)",fontsize=15)
            elif jj == 4:
                ax.set_ylabel("Mean Error (truth - prediction) [W/m$^2$]",fontsize=14)
                ax.set_ylim(-0.2,0.2)
                ax.set_title("(e)",fontsize=15)
    

# Get model settings
settings = experiment_settings.get_settings("MPI_IV_hpt003")

# Load predictions
settings["rng_seed"] = 3
model_name = file_methods.get_model_name(settings)
predictions = file_methods.load_predictions(PREDICTIONS_DIRECTORY + model_name + "_predictions.pickle")

label5, preds5, label95, preds95 = get_percentiles(predictions["labels_val"], predictions["pred_val"], th=percentile_th)

ax = plt.subplot(2, 3, 6)
ax.scatter(predictions["labels_val"], predictions["pred_val"],s=2,color='k',label="All validation members")
ax.scatter(label5, preds5,s=2,color='r',label="Below 5th percentile")
ax.scatter(label95, preds95,s=2,color='g',label="Above 95th percentile")
ax.plot([-1.5,1.5],[-1.5,1.5],'k--')
ax.set_xlim(-1.3,1.3)
ax.set_ylim(-1.3,1.3)
ax.set_xlabel("Truth [W/m$^2$]",fontsize=14)
ax.set_ylabel("Prediction [W/m$^2$]",fontsize=14)
ax.set_title("(f)",fontsize=15)
ax.legend(fontsize=14,markerscale=3,frameon=False)
